## Web Crawler → SQLite3 on eBay

### About this project
- This project is mix of using web-crawling, data cleaning, and storing data into a database
- Here, I will be using the following libraries:
    - pandas
    - re
    - beautifulsoup
    - requests
    - sqlite3
- I will also be commenting on places where you can  change the variables to adjust to your needs

#### Steps
- Create a funtions that stores your "keyword" (item you want on eBay)
- Use the search engine on ebay website to search your "keyword"
    - get the first n pages
- Extract the items' information
    - title of the item
    - price
    - condition 
    - shipping information
    - returning policy

|keyword|rank|item_title|price|condition|shipping_info|return|
|:--|:--|:--|:--|:--|:--|:--|
|starwars|1||The Black Series Princess|9.99|brand new|free|null|
|starwars|2|Luke Skywalker & Ysalamiri |19.99|brand new|free|free|
|starwars|3|The Black Series Luke Skywalker|25.99|brand new|5.99|null|
|...|...|...|...|...|...|...|
|...|...|...|...|...|...|...|
|lego|1|100p lego |19.99|pre-owned|free|null|

- Store the data above into a database using SQLite3
    - write SQL to analyze the data

## All the importing libraries

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3
import re
import warnings
warnings.filterwarnings('ignore')

## STEP 1
#### Making a list of keywords

In [2]:
keyword = input('Type in words that you want to search: ')
keys = keyword.split(' ')

In [3]:
keys

['']

## STEP 2
#### Create a web crawler on Yahoo search engine to extract information

In [4]:

keywords = ["starwars", "lego"]

title_box = []
price_box = []
condition_box = []
shipping_box = []
return_box = []

for k in keywords:
    for pn in range(1, 4):
        URL = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw={0}&_pgn={1}'.format(k,pn)
        res = requests.get(URL)
        if res.status_code != 200:
            continue
        res.coding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        elms = soup.find_all('div', {'class': 's-item__info clearfix'})
        for i, elm in enumerate(elms):
            title = elm.find_all('h3', {'class': 's-item__title'})[0].get_text()
            title_box.append({'keyword': k, 'page': pn, 'index': i, 'title': title})
            try:
                price = elm.find_all('span', {'class': 's-item__price'})[0].get_text()
            except:
                continue
            price_box.append({'keyword': k, 'page': pn, 'index': i, 'price': price})
            try:
                cond = elm.find_all('span', {'class': 'SECONDARY_INFO'})[0].get_text()
            except:
                continue
            condition_box.append({'keyword': k, 'page': pn, 'index': i, 'condition': cond})
            try:
                ship = elm.find_all('span', {'class': 's-item__shipping s-item__logisticsCost'})[0].get_text()
            except:
                continue
            shipping_box.append({'keyword': k, 'page': pn, 'index': i, 'shipping': ship})
            try:
                r = elm.find_all('span', {'class': 's-item__free-returns s-item__freeReturnsNoFee'})[0].get_text()
            except:
                continue
            return_box.append({'keyword': k, 'page': pn, 'index': i, 'return': r})
            

## STEP 3
#### From the dictionaries we made from above, store it in a data frame

In [5]:
# functions for all 5 description (columns or features)
def todf(x):
    df = pd.DataFrame.from_dict(x[0], orient = 'index').T
    for k in range(1, len(x)):
        df = df.append(x[k], ignore_index = True)
    return df

In [6]:
title_df = todf(title_box)
price_df = todf(price_box)
cond_df = todf(condition_box)
ship_df = todf(shipping_box)
r_df = todf(return_box)
DF = pd.merge(title_df, price_df, on=['keyword', 'page', 'index'], how='left')
DF = pd.merge(DF, cond_df, on=['keyword', 'page', 'index'], how='left')
DF = pd.merge(DF, ship_df, on=['keyword', 'page', 'index'], how='left')
DF = pd.merge(DF, r_df, on=['keyword', 'page', 'index'], how='left')

In [7]:
# In the price column, takeaway the $ and ',' and  if there is a range of values, take the average
# return Nan for other non-numeric values
def price_norm(x):
    try:
        x = x.replace('$', '').replace(',', '')
        if 'to' in x:
            f = x.split(' to ')[0]
            s = x.split(' to ')[1]
            ans = (float(f) + float(s))/2
            return ans
        return float(x)
    except:
        return np.nan

# In the shipping column, cahnge it to the following: Free shipping = free, Nan = Nan, +$7.75 shipping = 7.75
def ship_norm(y):
    y = str(y).replace(' ', '')
    if 'Free' in y:
        return 'free'
    elif '+$' in y:
        y = y.replace('+$', '').replace('shipping', '')
        return float(y)
    else:
        return y

# In the shipping column, cahnge it to the following: Free shipping = free, Nan = Nan, +$7.75 shipping = 7.75
def return_norm(z):
    if 'Free' in str(z):
        return 'free'
    else:
        return z


In [8]:
test = DF

In [9]:
# just drop the Nan in price column
test  = test[test['price'].notna()]

# creating a new index from 1 to the length of the current dataframe
test.index = np.arange(1, len(test)+1)

# clean the price, shipping, and return column
test['Price'] = test.price.apply(lambda x: price_norm(x))
test['Shipping'] = test.shipping.apply(lambda y : ship_norm(y))
test['Return'] = test['return'].apply(lambda z : return_norm(z))

# make the index to rank column
test['Rank'] = test.index

# drop and reorder the columns
test = test.drop(columns = ['page', 'index', 'price', 'condition', 'shipping', 'return'])
test = test.reindex(columns=['keyword','Rank', 'title', 'Price', 'Shipping', 'Return'])
test

,keyword,Rank,title,Price,Shipping,Return
1,starwars,1,Star Wars Black Series Mandalorian Beskar Armo...,28.98,free,free
2,starwars,2,STAR WARS THE VINTAGE COLLECTION THE MANDALORI...,18.95,free,NaN
3,starwars,3,"Star Wars Mandalorian The Child 11"" Plush Baby...",29.99,free,free
4,starwars,4,Star Wars Black Series The Bad Batch Exclusive...,46.99,free,NaN
5,starwars,5,Star Wars The Black Series Han Solo 6-Inch-Sca...,26.49,free,NaN
...,...,...,...,...,...,...
386,lego,386,New ListingLEGO 21162 MINECRAFT THE TAIGA ADVE...,19.99,free,NaN
387,lego,387,New ListingLego 4727-1 Aragog in the Dark Fore...,39.99,16.1,NaN
388,lego,388,LEGO Sunflowers 40524 PRESALE - FREE EXPEDITED...,45.00,free,NaN
389,lego,389,Lego Tile 2 x 4 Flat Smooth Parts Pieces Lot A...,16.45,free,NaN


## STEP 4
#### Create a function that will return a dataframe from inputting an arbritary keyword or keywords. (Combine STEP 1~3)

In [10]:
# all of the functions needed to process this algorithm
##########################################
# functions for all 5 description (columns or features)
def todf(x):
    df = pd.DataFrame.from_dict(x[0], orient = 'index').T
    for k in range(1, len(x)):
        df = df.append(x[k], ignore_index = True)
    return df

# In the price column, takeaway the $ and  if there is a range of values, take the average
def price_norm(x):
    try:
        x = x.replace('$', '').replace(',', '')
        if 'to' in x:
            f = x.split(' to ')[0]
            s = x.split(' to ')[1]
            ans = (float(f) + float(s))/2
            return ans
        return float(x)
    except:
        return np.nan

# In the shipping column, cahnge it to the following: Free shipping = free, Nan = Nan, +$7.75 shipping = 7.75
def ship_norm(y):
    y = str(y).replace(' ', '').replace(',', '')
    if 'Free' in y:
        return 'free'
    elif '+$' in y:
        y = y.replace('+$', '').replace('shipping', '')
        return float(y)
    else:
        return y

# In the shipping column, cahnge it to the following: Free shipping = free, Nan = Nan, +$7.75 shipping = 7.75
def return_norm(z):
    if 'Free' in str(z):
        return 'free'
    else:
        return z


# create a function called search_table
def search_table(keywords, page = 4):
    title_box = []
    price_box = []
    condition_box = []
    shipping_box = []
    return_box = []

    for k in keywords:
        for pn in range(1, page):
            URL = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw={0}&_pgn={1}'.format(k,pn)
            res = requests.get(URL)
            if res.status_code != 200:
                continue
            res.coding = 'utf-8'
            soup = BeautifulSoup(res.text, 'html.parser')
            elms = soup.find_all('div', {'class': 's-item__info clearfix'})
            for i, elm in enumerate(elms):
                title = elm.find_all('h3', {'class': 's-item__title'})[0].get_text()
                title_box.append({'keyword': k, 'page': pn, 'index': i, 'title': title})
                try:
                    price = elm.find_all('span', {'class': 's-item__price'})[0].get_text()
                except:
                    continue
                price_box.append({'keyword': k, 'page': pn, 'index': i, 'price': price})
                try:
                    cond = elm.find_all('span', {'class': 'SECONDARY_INFO'})[0].get_text()
                except:
                    continue
                condition_box.append({'keyword': k, 'page': pn, 'index': i, 'condition': cond})
                try:
                    ship = elm.find_all('span', {'class': 's-item__shipping s-item__logisticsCost'})[0].get_text()
                except:
                    continue
                shipping_box.append({'keyword': k, 'page': pn, 'index': i, 'shipping': ship})
                try:
                    r = elm.find_all('span', {'class': 's-item__free-returns s-item__freeReturnsNoFee'})[0].get_text()
                except:
                    continue
                return_box.append({'keyword': k, 'page': pn, 'index': i, 'return': r})
    title_df = todf(title_box)
    price_df = todf(price_box)
    cond_df = todf(condition_box)
    ship_df = todf(shipping_box)
    r_df = todf(return_box)
    DF = pd.merge(title_df, price_df, on=['keyword', 'page', 'index'], how='left')
    DF = pd.merge(DF, cond_df, on=['keyword', 'page', 'index'], how='left')
    DF = pd.merge(DF, ship_df, on=['keyword', 'page', 'index'], how='left')
    DF = pd.merge(DF, r_df, on=['keyword', 'page', 'index'], how='left')
    DF  = DF[DF['price'].notna()]
    DF.index = np.arange(1, len(DF)+1)
    DF['Price'] = DF.price.apply(lambda x: price_norm(x))
    DF['Shipping'] = DF.shipping.apply(lambda y : ship_norm(y))
    DF['Return'] = DF['return'].apply(lambda z : return_norm(z))
    DF['Rank'] = DF.index
    DF = DF.drop(columns = ['page', 'index', 'price', 'condition', 'shipping', 'return'])
    DF = DF.reindex(columns=['keyword','Rank', 'title', 'Price', 'Shipping', 'Return'])
    return DF

In [16]:
#input keywords
keywords = input('Type in words that you want to search (if you want to do an AND search, put "+" between the words): ')
# input will be "iphone samsung dell intel LG" for this example

In [17]:
keywords = keywords.split(' ')


In [18]:
df = search_table(keywords)
df.head(20)

,keyword,Rank,title,Price,Shipping,Return
1,iphone,1,Apple iPhone 8 64GB 256GB Unlocked Straight ta...,234.00,free,free
2,iphone,2,Apple iphone 7/7 plus 32GB/128GB Black Gold Un...,140.00,free,free
3,iphone,3,Apple iPhone X 64GB/256GB Gray Silver Unlocked...,303.50,free,free
4,iphone,4,New ListingApple iPhone 7 Plus - 256GB - Jet B...,150.00,free,NaN
5,iphone,5,New ListingApple iPhone XR - 64GB - Black (AT&...,285.00,11.6,NaN
6,iphone,6,New ListingApple iPhone 12 - 128GB - Black - U...,499.00,free,NaN
7,iphone,7,New ListingApple iPhone X - 64GB - Space Gray ...,56.00,free,NaN
8,iphone,8,Apple iPhone 8 64GB 256GB Unlocked Straight ta...,259.00,free,NaN
9,iphone,9,Apple iPhone 8 Plus 256gb Unlocked Straight ta...,349.00,free,NaN
10,iphone,10,Apple iPhone XR 64GB Factory Unlocked Smartpho...,299.99,free,NaN


In [19]:
df.shape

(1005, 6)

## STEP 5
#### After making the complete table, import it into a database using sqlite

In [20]:
# connecting to sqlite
conn = sqlite3.connect("ebay_item_lists")
df.to_sql('ebay_tables', conn, if_exists = 'append', index = None)
conn.close()

### Queries to think about
- What percentage of the items have free shipping
- What percentage of the items have free returning policy
- What are the range of prices for every keyword

In [29]:
## What percentage of the items have free shipping
conn = sqlite3.connect("ebay_item_lists")

first = pd.read_sql('''
        SELECT
            keyword,
            sum(CASE WHEN Shipping = "free" THEN 1 ELSE 0 END) as free_cnt,
            count(*) as total
        FROM
            ebay_tables
        GROUP BY
            keyword
        ORDER BY
            keyword''', con = conn)

conn.close()

In [30]:
first

,keyword,free_cnt,total
0,LG,417,615
1,dell,370,613
2,intel,402,585
3,iphone,332,615
4,samsung,405,615


In [33]:
first['free_shipping_percent'] = first['free_cnt']/first['total']
first

,keyword,free_cnt,total,free_shiiping_percent,free_shipping_percent
0,LG,417,615,0.678049,0.678049
1,dell,370,613,0.603589,0.603589
2,intel,402,585,0.687179,0.687179
3,iphone,332,615,0.539837,0.539837
4,samsung,405,615,0.658537,0.658537


In [34]:
## What percentage of the items have free returning policy
conn = sqlite3.connect("ebay_item_lists")

second = pd.read_sql('''
        SELECT
            keyword,
            sum(CASE WHEN Return = "free" THEN 1 ELSE 0 END) as free_cnt,
            count(*) as total
        FROM
            ebay_tables
        GROUP BY
            keyword
        ORDER BY
            keyword''', con = conn)

conn.close()

In [36]:
second['free_return_percent'] = second['free_cnt']/second['total']
second

,keyword,free_cnt,total,free_return_percent
0,LG,333,615,0.541463
1,dell,252,613,0.411093
2,intel,182,585,0.311111
3,iphone,273,615,0.443902
4,samsung,321,615,0.521951


In [25]:
## What are the range of prices for every keyword
conn = sqlite3.connect("ebay_item_lists")

third = pd.read_sql('''
        SELECT
            keyword,
            min(Price) as min_price,
            max(Price) as max_price,
            max(Price) - min(Price) as range
        FROM
            ebay_tables
        GROUP BY
            keyword
        ORDER BY
            keyword''', con = conn)

conn.close()

In [26]:
third

,keyword,min_price,max_price,range
0,LG,0.99,699.99,699.00
1,dell,0.99,1079.99,1079.00
2,intel,0.01,2250.00,2249.99
3,iphone,0.01,1699.99,1699.98
4,samsung,0.99,1125.99,1125.00
